In [3]:
import numpy as np
import pandas as pd
train_data = pd.read_csv('/kaggle/input/toxicity/train.csv')
train_data['labels'] = train_data[['toxic','severe_toxic','obscene','threat','insult','identity_hate']].values.tolist()
test_data = pd.read_csv('/kaggle/input/toxicity/test.csv')
test_labels = pd.read_csv('/kaggle/input/toxicity/test_labels.csv')

In [ ]:
#test_labels = test_labels.drop('id',axis =1)
import pandas as pd
import torch
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import DistilBertTokenizer, DistilBertModel


In [4]:
weights = [1,2,1,1,1,1]
toxicity = []
for i in train_data['labels'].values:
    weighted = 0
    for j in range(len(i)):
        weighted += i[j] * weights[j]
        w_s = weighted/sum(weights)
    toxicity.append(w_s)

In [5]:
toxicity_reshaped = []
for i in toxicity:
    if(i ==0):
        toxicity_reshaped.append(0)
    else:
        toxicity_reshaped.append(1)

In [ ]:
count_zero, count_one = 0, 0
for i in toxicity_reshaped:
    if(i == 0):
        count_zero += 1
    else:
        count_one += 1
print(count_zero,count_one)
#157209 2362 0.5 threshold
#153145 6426 0.3 threshold
#149706 9865 0.2 threshold
#143346 16225 0 threshold
print(toxicity_reshaped)

In [7]:
!pip install -q transformers


In [21]:
###### Preprocessing - removing https links and ip address #####
import re
removed_hyperlinks_training = []
removed_hyperlinks_testing = []
pattern = re.compile(r"(http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+)|(\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3})")

for val in train_data['comment_text'].values:
  temp = re.sub(pattern,'',val)
  removed_hyperlinks_training.append(temp)

for val in test_data['comment_text'].values:
  temp = re.sub(pattern,'',val)
  removed_hyperlinks_testing.append(temp)


In [22]:
#### Preprocessing - removing html tags and code ######
tags = re.compile(r"\|.*")
removed_html_tags_code_training = []
removed_html_tags_code_testing = []
for val in removed_hyperlinks_training:
  temp = re.sub(tags,'',val)
  removed_html_tags_code_training.append(temp)

for val in removed_hyperlinks_testing:
  temp = re.sub(tags,'',val)
  removed_html_tags_code_testing.append(temp)

In [23]:
### Preprocessing - removing numbers, digits and punctuations except “”’, “.”, “!”, “?”. ######
removed_numbers_punctuations_training = []
removed_numbers_punctuations_testing = []
for val in removed_html_tags_code_training:
    temp = re.sub(r"\d+", "", val)
    temp = re.sub(r"[^\w\s''-]", "",temp) #r"[^\w\s.!?''-]"
    removed_numbers_punctuations_training.append(temp)

for val in removed_html_tags_code_testing:
    temp = re.sub(r"\d+", "", val)
    temp = re.sub(r"[^\w\s''-]", "",temp) #r"[^\w\s.!?''-]"
    removed_numbers_punctuations_testing.append(temp)

In [24]:
## Removing Stopwords
stopwords_english = ['then', 'only', 'shouldn', 'themselves', "hadn't", 'off', 'm', "isn't", 'because', 'couldn', 'most', 'any', 'not', 'what', 'yours', 'very', 'them', 'there', "don't", "you're", 'that', "wouldn't", 'd', 'did', 'further', 'own', 'needn', 'they', 'my', 'o', 'mustn', 'didn', 'as', 'doesn', "she's", 'ain', 'will', 'too', "hasn't", 'to', 'a', 'aren', 'i', 'who', 'those', 'or', 'while', 'here', 'no', "didn't", 'during', 'until', 'before', 'yourselves', 'same', 'itself', 'you', 'am', "doesn't", 'these', 'll', 'which', 'was', 'mightn', 'more', "it's", 'do', 'all', 'has', 'won', 'once', 'isn', 'had', 'haven', 'at', 'yourself', 'herself', 'about', 'how', 'ourselves', 'this', "you'll", 'theirs', 'were', "mightn't", 'few', 'being', 'having', 'out', 'wouldn', 'now', 'him', 'by', 'through', 'against', 'nor', 'of', 'should', 'some', "haven't", 'up', 'when', "aren't", 'hadn', "that'll", 'so', 'ours', 'does', 'weren', 'he', 'and', 'into', 'be', 'from', "weren't", 'but', 'wasn', "couldn't", 'our', 'can', 'after', 'his', 'other', 'their', 'where', 'with', 'are', "needn't", 'its', 'than', 'her', 'she', 'over', "shan't", 've', 'down', 'on', 'again', 't', 'below', 'in', 'have', "wasn't", 'y', 'for', 'hasn', 'between', "mustn't", 'himself', 'shan', 'the', 'an', 'both', 'is', 'doing', 'just', "should've", 'your', 'don', 'myself', 'above', 'such', "you've", 'it', 'been', 'me', "won't", "you'd", 'each', 's', 'we', 're', 'hers', 'ma', 'why', "shouldn't", 'whom', 'under', 'if']

In [25]:
######### tokenize the corpus ###############
tokenized_cleaned_comments_training = []
tokenized_cleaned_comments_testing = []
for i in removed_numbers_punctuations_training:

    temp_text = re.split('(\W)', i)
    temp_text = [j for j in temp_text if j not in r"[^a-zA-Z']+|[0-9]+|[.]+|\s+ " ]
    temp_text = [j for j in temp_text if j.lower() not in stopwords_english]
    tokenized_cleaned_comments_training.append(temp_text)

for i in removed_numbers_punctuations_testing:

    temp_text = re.split('(\W)', i)
    temp_text = [j for j in temp_text if j not in r"[^a-zA-Z']+|[0-9]+|[.]+|\s+ " ]
    temp_text = [j for j in temp_text if j.lower() not in stopwords_english]
    tokenized_cleaned_comments_testing.append(temp_text)

In [37]:
cleaned_trainig_data = [' '.join(tokens).replace('\n',"") for tokens in tokenized_cleaned_comments_training]
cleaned_trainig_test = [' '.join(tokens).replace('\n',"") for tokens in tokenized_cleaned_comments_testing]

In [43]:
label_list, comment_list = [], []
for comment, label in zip(cleaned_trainig_data, train_data[['toxic','severe_toxic','obscene','threat','insult','identity_hate']].values):
    label = label.tolist()
    comment_list.append(comment)
    label_list.append(label)

final_cleaned_dataset_train = pd.DataFrame({'Comment': comment_list, 'Label': label_list})

In [48]:
final_cleaned_dataset_train =  final_cleaned_dataset_train.sample(frac = 1)

In [53]:
maxilen = 128
train_batch = 32
val_batch = 32
dev = 'cuda:0' if torch.cuda.is_available() else 'cpu'
print(dev)

cuda:0


In [55]:
class ToxicComments_dataset(Dataset):

    def __init__(self, df, token, max_len, new_data=False):
        self.tokenizer = token
        self.data = df
        self.text = df.Comment
        self.new_data = new_data
        
        if not new_data:
            self.targets = self.data.Label
        self.max_len = max_len

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        text = str(self.text[index])
        text = " ".join(text.split())

        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]

        out = {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
        }
        
        if not self.new_data:
            out['targets'] = torch.tensor(self.targets[index], dtype=torch.float)

        return out


In [56]:
train_size = 0.9

train_data_complete = final_cleaned_dataset_train.sample(frac=train_size, random_state=123)
val_data_complete = final_cleaned_dataset_train.drop(train_data_complete.index).reset_index(drop=True)
train_data_complete = train_data_complete.reset_index(drop=True)


print("Orig Dataset: {}".format(train_data.shape))
print("Training Dataset: {}".format(train_data_complete.shape))
print("Validation Dataset: {}".format(val_data_complete.shape))

distil_tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased', truncation=True, do_lower_case=True)
training_set = ToxicComments_dataset(train_data_complete, distil_tokenizer, maxilen)
val_set = ToxicComments_dataset(val_data_complete, distil_tokenizer, maxilen)

Orig Dataset: (159571, 9)
Training Dataset: (143614, 2)
Validation Dataset: (15957, 2)


In [58]:
train_params = {'batch_size': 32,
                'shuffle': True,
                'num_workers': 8
                }

val_params = {'batch_size': 32,
               'shuffle': True,
               'num_workers': 8
                }

train_data_load = DataLoader(training_set, **train_params)
val_data_load = DataLoader(val_set, **val_params)

/opt/conda/lib/python3.10/site-packages/torch/utils/data/dataloader.py:561: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [61]:
class DistilBERTClass(torch.nn.Module):
    def __init__(self):
        super(DistilBERTClass, self).__init__()
        
        self.bert = DistilBertModel.from_pretrained("distilbert-base-uncased")
        self.classifier = torch.nn.Sequential(
            torch.nn.Linear(768, 768),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.1),
            torch.nn.Linear(768, 6)
        )

    def forward(self, input_ids, attention_mask, token_type_ids):
        output_1 = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        hidden_state = output_1[0]
        out = hidden_state[:, 0]
        out = self.classifier(out)
        return out

distil_model = DistilBERTClass()
distil_model.to(dev);

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [62]:
optimizer = torch.optim.Adam(params=distil_model.parameters(), lr=1e-5)

In [64]:
def train(epoch):
    model.train()
    
    for i, data in tqdm(enumerate(train_data_load, 0)):
        ids = data['ids'].to(dev, dtype=torch.long)
        mask = data['mask'].to(dev, dtype=torch.long)
        token_type_ids = data['token_type_ids'].to(dev, dtype=torch.long)
        targets = data['targets'].to(dev, dtype=torch.float)

        outputs = distil_model(ids, mask, token_type_ids)

        optimizer.zero_grad()
        loss = torch.nn.functional.binary_cross_entropy_with_logits(outputs, targets)
        
        if i % 2500 == 0:
            print(f'Epoch: {epoch}, Loss:  {loss.item()}')
        
        loss.backward()
        optimizer.step()

In [65]:
for i in range(3):
    train(i)

/opt/conda/lib/python3.10/site-packages/torch/utils/data/dataloader.py:561: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to m

Epoch: 0, Loss:  0.7104593515396118


2501it [13:36,  3.07it/s]

Epoch: 0, Loss:  0.06430947780609131


4488it [24:24,  3.06it/s]
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2377: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest

...

4488it [24:24,  3.06it/s]
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2377: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest

Epoch: 2, Loss:  0.008854628540575504


2501it [13:36,  3.06it/s]

Epoch: 2, Loss:  0.004425479099154472


4488it [24:25,  3.06it/s]


In [66]:
test_labels = pd.read_csv('/kaggle/input/toxicity/test_labels.csv')

In [68]:
test_labels = test_labels.drop('id',axis = 1)

In [72]:
comment_list_test = []
for comment in (cleaned_trainig_test):
    #label = label.tolist()
    comment_list_test.append(comment)
    #label_list.append(label)

final_cleaned_dataset_test = pd.DataFrame({'Comment': comment_list_test})

In [77]:
test_params = {'batch_size': 32,
               'shuffle': True,
               'num_workers': 8
                }

test_set = ToxicComments_dataset(final_cleaned_dataset_test, distil_tokenizer, maxilen, new_data=True)
test_loader = DataLoader(test_set, **test_params)

/opt/conda/lib/python3.10/site-packages/torch/utils/data/dataloader.py:561: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [78]:
predictions = []

def test(epoch):
    model.eval()
    
    with torch.inference_mode():
    
        for i, text in tqdm(enumerate(test_loader, 0)):


            ids = text['ids'].to(dev, dtype=torch.long)
            mask = text['mask'].to(dev, dtype=torch.long)
            token_type_ids = text['token_type_ids'].to(dev, dtype=torch.long)
            outputs = model(ids, mask, token_type_ids)
            probaility_outputs = torch.sigmoid(outputs)

            predictions.append(probaility_outputs)
            
            
    return probaility_outputs
probaility_outputs = test(distil_model)

/opt/conda/lib/python3.10/site-packages/torch/utils/data/dataloader.py:561: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to m

In [91]:
torch.save(distil_model,'final_distil_model.h5')

In [85]:
complete_predictions = torch.cat(predictions)

In [88]:
print(len(complete_predictions))
print(complete_predictions)

153164
tensor([[0.4854, 0.4607, 0.4746, 0.4781, 0.5186, 0.5342],
        [0.4815, 0.4629, 0.4874, 0.4888, 0.5155, 0.5277],
        [0.4857, 0.4656, 0.4883, 0.4996, 0.4750, 0.5073],
        ...,
        [0.4887, 0.4598, 0.4923, 0.5085, 0.5028, 0.4958],
        [0.4919, 0.4666, 0.4852, 0.4946, 0.4883, 0.5242],
        [0.4839, 0.4427, 0.4885, 0.5087, 0.4988, 0.5083]], device='cuda:0')


In [94]:
final_submissions_dataset = test_data.copy()
final_submissions_dataset.drop("comment_text", inplace=True, axis=1)

In [95]:
for i,name in enumerate(label_columns):

    final_submissions_dataset[name] = complete_predictions[:, i].cpu()
    #final_submissions_dataset.head()

In [96]:
final_submissions_dataset

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,0.485438,0.460746,0.474592,0.478135,0.518604,0.534152
1,0000247867823ef7,0.481509,0.462905,0.487369,0.488781,0.515530,0.527711
2,00013b17ad220c46,0.485672,0.465597,0.488295,0.499621,0.475014,0.507290
3,00017563c3f7919a,0.476427,0.461015,0.485920,0.488724,0.496532,0.520164
4,00017695ad8997eb,0.482547,0.449387,0.475115,0.486569,0.510244,0.520836
...,...,...,...,...,...,...,...
153159,fffcd0960ee309b5,0.486624,0.463431,0.470220,0.488230,0.505266,0.520939
153160,fffd7a9a6eb32c16,0.485365,0.458224,0.485186,0.485934,0.495648,0.531145
153161,fffda9e8d6fafa9e,0.488651,0.459759,0.492317,0.508490,0.502825,0.495790
153162,fffe8f1340a79fc2,0.491879,0.466649,0.485157,0.494597,0.488288,0.524162


In [98]:
final_submissions_dataset.to_csv('submission_distil_final.csv', index=False)